In [1]:
# Importing packages for analyzing useses
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
from datetime import datetime
from datetime import timedelta

register_matplotlib_converters()
sns.set()

In [2]:
# Import the data and making a dataframe out of it
file_name = '../Data/dataGM_excel2.xlsx'
productionDF = pd.read_excel(file_name, sheet_name='Productie')
consumptionDF = pd.read_excel(file_name, sheet_name='Consumptie')

df_prod = productionDF.iloc[4:]
df_cons = consumptionDF.iloc[4:]

display(df_prod.head())
display(df_cons.head())

,ID-nummer,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H24,H25,H26,H27,H28,H29,H30,H31,H32,H33
4,2017-07-11 00:15:00,0,0,0,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2017-07-11 00:30:00,0,0,0,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,2017-07-11 00:45:00,0,0,0,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,2017-07-11 01:00:00,0,0,0,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2017-07-11 01:15:00,0,0,0,0,NaN,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,ID-nummer,H01,H02,H03,H04,H05,H06,H07,H08,H09,...,H24,H25,H26,H27,H28,H29,H30,H31,H32,H33
4,2017-07-11 00:15:00,0.038,0.039,0.044,0.024,NaN,0.039,0.025,0.006,0.294,...,0.053,0.018,0.076,0.07,0.036,0.014,0.044,0.416,0.023,0.053
5,2017-07-11 00:30:00,0.018,0.027,0.034,0.024,NaN,0.051,0.017,0.007,0.165,...,0.049,0.019,0.083,0.061,0.027,0.026,0.034,0.17,0.031,0.029
6,2017-07-11 00:45:00,0.028,0.014,0.026,0.018,NaN,0.076,0.018,0.006,0.107,...,0.043,0.018,0.292,0.054,0.018,0.011,0.038,0.075,0.025,0.028
7,2017-07-11 01:00:00,0.026,0.014,0.474,0.021,NaN,0.08,0.016,0.006,0.105,...,0.058,0.019,0.049,0.034,0.022,0.026,0.032,0.099,0.024,0.041
8,2017-07-11 01:15:00,0.018,0.014,0.084,0.117,NaN,0.438,0.023,0.006,0.103,...,0.045,0.018,0.048,0.208,0.015,0.017,0.043,0.054,0.025,0.032


In [3]:
def fill_blank_outliers(df):
    df_local = df.copy()
    blank_indexes = df.isnull().stack()[lambda x: x].index.tolist()
    
    outliers_indexes = []
    
    for pos in blank_indexes:
        outliers_indexes.append((pos[0] + 1, pos[1]))
        
    blank_indexes_set = set(blank_indexes)
    outliers_indexes_set = set(outliers_indexes)
    
    final_indexes = list(outliers_indexes_set - blank_indexes_set)
    
    df_outliers = pd.DataFrame(final_indexes, columns=['row', 'col'])
    
    for index in final_indexes:
        df_local.loc[index[0]][index[1]] = 0
        
    df_final = df_local.copy()
    df_final.fillna(0, inplace=True)
    
    return df_final

In [4]:
df_prod_clean = fill_blank_outliers(df_prod)
df_cons_clean = fill_blank_outliers(df_cons)

In [5]:
df_prod_clean_kwh = df_prod_clean.copy()
df_prod_clean_kwh_trans = df_prod_clean_kwh
df_prod_clean_kwh_trans = df_prod_clean_kwh_trans.set_index('ID-nummer')

df_cons_clean_kwh = df_cons_clean.copy()
df_cons_clean_kwh_trans = df_cons_clean_kwh
df_cons_clean_kwh_trans = df_cons_clean_kwh_trans.set_index('ID-nummer')

In [6]:
# Function to extract the max values from a dataset
def extractMaxValues(df, group='D'):
    if (group != '15Min'):
        df_max_values = pd.DataFrame(df.groupby(pd.Grouper(freq=group)).sum())
    else:
        df_max_values = df.max() # Max values per house
        df_max_values = df[df[df_max_values.index.tolist()] == df_max_values.tolist()] # DataFrame with max values
        df_max_values = df_max_values.dropna(how='all') # DataFrame cleaned

        df_indexes = pd.DataFrame() # List with the indexes of the max values

        for idx, row in df_max_values.iterrows(): # For every row...
            for value in row: # For every value...
                if (pd.notna(value)): # If there is a number...
                    if (len(df_indexes) == 0):
                        df_indexes = df_indexes.append({'date': idx, 'house': row[row == value].index[0], 'value': value}, ignore_index=True) # The index is added to the list
                        df_indexes = df_indexes.sort_values('house')
                    elif (row[row == value].index[0] not in df_indexes['house'].values):
                        df_indexes = df_indexes.append({'date': idx, 'house': row[row == value].index[0], 'value': value}, ignore_index=True) # The index is added to the list
                        df_indexes = df_indexes.sort_values('house')
        
        return df_indexes
    
    return df_max_values

In [7]:
df_house_prod_clean_max = extractMaxValues(df_prod_clean_kwh_trans, 'D')
df_house_cons_clean_max = extractMaxValues(df_cons_clean_kwh_trans, 'D')

df_house_prod_clean_max_15min = extractMaxValues(df_prod_clean_kwh_trans, '15Min')
df_house_cons_clean_max_15min = extractMaxValues(df_cons_clean_kwh_trans, '15Min')

In [8]:
def calculateAmperes(kwh, voltage):
    watts = kwh / (1/4) * 1000
    amperes = watts / voltage
    return amperes

In [9]:
def createTable(df_prod, df_cons, df_prod_15, df_cons_15):
    df_max = pd.DataFrame()
    df_max['max_prod_date'] = df_prod.idxmax()
    df_max['max_prod'] = df_prod.max()
    df_max['max_cons_date'] = df_cons.idxmax()
    df_max['max_cons'] = df_cons.max()
    
    df_max['max_prod_amperes_date'] = df_prod_15['date'].values
    df_max['max_prod_amperes'] = calculateAmperes(df_prod_15['value'].values, 230)
    df_max['max_cons_amperes_date'] = df_cons_15['date'].values
    df_max['max_cons_amperes'] = calculateAmperes(df_cons_15['value'].values, 230)
    
    return df_max

## Amperes Table

In [10]:
# Table with max values
df_table_max = createTable(df_house_prod_clean_max,
                           df_house_cons_clean_max,
                           df_house_prod_clean_max_15min,
                           df_house_cons_clean_max_15min)
df_table_max

,max_prod_date,max_prod,max_cons_date,max_cons,max_prod_amperes_date,max_prod_amperes,max_cons_amperes_date,max_cons_amperes
H01,2018-07-02,32.651,2018-03-01,59.567,2018-07-02 13:45:00,15.843478,2017-12-11 18:00:00,36.243478
H02,2018-06-26,22.968,2017-12-31,61.718,2019-05-05 13:30:00,13.443478,2018-02-17 09:30:00,33.130435
H03,2018-07-05,15.145,2018-03-03,25.126,2019-05-12 13:15:00,8.800000,2017-10-03 18:15:00,24.086957
H04,2018-06-26,20.181,2017-12-29,21.525,2018-06-21 14:00:00,11.130435,2019-01-20 21:45:00,17.495652
H05,2018-06-13,19.169,2019-01-21,19.637,2018-06-21 14:00:00,12.139130,2018-10-20 18:15:00,23.286957
H06,2019-05-14,22.617,2018-01-13,40.147,2019-05-12 13:15:00,13.391304,2017-12-13 18:30:00,33.286957
H07,2018-06-26,24.840,2018-03-01,75.839,2018-05-21 13:45:00,13.686957,2018-03-01 19:45:00,45.321739
H08,2018-06-26,14.688,2017-12-31,66.913,2018-06-21 14:00:00,8.121739,2017-11-25 08:15:00,38.121739
H09,2018-07-03,26.461,2017-12-29,60.675,2017-10-12 12:30:00,17.286957,2018-02-24 17:30:00,37.165217
H10,2019-05-24,22.049,2018-12-15,93.523,2019-04-13 13:00:00,12.852174,2019-03-16 07:15:00,43.686957
